In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.3 MB 26.8 MB/s 
     |████████████████████████████████| 15.7 MB 45.7 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=14c2700404c19d320b33d8a23e040b306d8c6e0475702f1da326398a44d133c8
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take in

# New Section

In [ ]:
pip install demoji

     |████████████████████████████████| 42 kB 710 kB/s 


In [ ]:
pip install openpyxl --upgrade


     |████████████████████████████████| 242 kB 5.2 MB/s 
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9


In [ ]:
import numpy as np
import pandas as pd
import json
import glob
from nltk.corpus import stopwords

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

import nltk
import re
nltk.download('stopwords')
import demoji
demoji.download_codes()
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

import spacy

import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/1 Pre_Event_En.xlsx')
df.head()

,index,author_id,name,username,location,id_location,created_at,lang,like_count,retweet_count,text,Longtitude,Latitude
0,1,2834847965,Regen-Alarm München,regenInMuenchen,München,1.414422e+18,2021-07-12T03:12:01.000Z,de,0,0,storm fog moderate today munich,11.525549,48.154890
1,2,1557947881,StephanSRT,StephanSRT,"Niedersachsen, Goslar",1.414459e+18,2021-07-12T05:36:32.000Z,de,4,0,you might think yesterday it rained heavily in...,10.443266,52.346401
2,3,19669684,Jan Ole Unger,jounger,"Hamburg, Germany",1.414469e+18,2021-07-12T06:17:08.000Z,de,2,0,"Incidentally, in puntaarenas there is the sout...",9.941648,53.567331
3,4,2315299130,Regen-Alarm Berlin,regenberlin,Berlin,1.414483e+18,2021-07-12T07:12:02.000Z,de,0,0,storm heat moderate morning berlin,13.397340,52.506786
4,5,2834830029,Regen-Alarm Köln,regenInKoeln,Köln,1.414500e+18,2021-07-12T08:20:02.000Z,de,0,0,storm continuous rain heavy morning thursday c...,6.955322,50.957555


In [ ]:
df['text'][1]

'you might think yesterday it rained heavily in Braunschweig'

#Data Cleaning

In [ ]:
# Remove URL
def remove_URL(text):
    url = re.compile(r'http\S+')
    return url.sub(r'',text)
df['text'] = df['text'].apply(lambda x : remove_URL(x))

In [ ]:
# Remove html tags

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

df['text']=df['text'].apply(lambda x : remove_html(x))

In [ ]:
# Remove emojis

def remove_emojis(text):
  new_text = demoji.replace(text, "")
  return new_text

df['text'] = df['text'].apply(lambda x: remove_emojis(x))

In [ ]:
# Remove mentions and hashtag

def remove_mention_hashtag(text):
    text=re.sub(r'@\w+','',text)        # remove mentions
    text=re.sub('[#]','',text)        # remove hashtag
    return text

df['text']=df['text'].apply(lambda x: remove_mention_hashtag(x))


In [ ]:
# Remove Emails

def remove_emails(text):
  email = re.compile(r'\S*@\S*\s?')
  return email.sub(r'', text)

df['text']=df['text'].apply(lambda x : remove_emails(x))

In [ ]:
# Remove new line characters

def remove_newline(text):
  newline = re.compile(r'\n+')
  return newline.sub(r'', text)

df['text']=df['text'].apply(lambda x : remove_newline(x))

In [ ]:
# Remove extra white space left while removing stuff

def remove_space(text):
    whitespace_remove = re.compile(r'\s+')
    return whitespace_remove.sub(r' ', text)

df["text"]=df["text"].apply(lambda x: remove_space(x))
len(df)

89

In [ ]:
punct=string.punctuation+'”‘’“'

def clean_text(text):
#Make text lowercase
    text = text.lower()
#remove text in square brackets
    text = re.sub(r'\[.*?\]', '', text)
#remove punctuation
    text = re.sub(r'[%s]' % re.escape(punct), '', text)
#remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)
    return text

tweets_df_clean = pd.DataFrame(df["text"].apply(lambda x: clean_text(x)))
data = tweets_df_clean['text'].to_list()

In [ ]:
# from deep_translator import GoogleTranslator


In [ ]:
len(data)

89

In [ ]:
df['text'] = data
# df2.dropna(inplace=True,axis=0)
df.replace(r'^\s*$', np.nan, regex=True,inplace = True)
# df2.dropna(inplace= True)
data_nan = df2['text']
for i in range(len(data_nan)):
    try:
        data_nan.iloc[i] = GoogleTranslator(source='auto', target='en').translate(data_nan.iloc[i])
    except:
        pass

In [ ]:
data[1]

'you might think yesterday it rained heavily in braunschweig'

# Lemmatization

In [ ]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(data)
print (lemmatized_texts[1])

may think yesterday rain heavily


# Stopwords

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = []
stopwords.extend(newStopWords)
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        new = [i for i in new if str(i) not in stopwords]
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[1])

['may', 'think', 'yesterday', 'rain', 'heavily']


# Bigram

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=20)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
data_words_bigrams = make_bigrams(data_words)
data_words = data_words_bigrams

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
id2word = corpora.Dictionary(data_words)

texts = data_words

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)
low_value = 0.03
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 1)]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=15)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.020604  0.106634       1        1  40.248504
0      0.093111 -0.054477       2        1  31.904175
1     -0.072507 -0.052157       3        1  27.847321, topic_info=                Term       Freq      Total Category  logprob  loglift
113   climate_crisis   6.000000   6.000000  Default  15.0000  15.0000
14             storm  16.000000  16.000000  Default  14.0000  14.0000
7       fire_brigade  10.000000  10.000000  Default  13.0000  13.0000
16             heavy   5.000000   5.000000  Default  12.0000  12.0000
112          climate   5.000000   5.000000  Default  11.0000  11.0000
..               ...        ...        ...      ...      ...      ...
30             water   2.000214   6.547932   Topic3  -4.5401   0.0925
15   continuous_rain   1.998934   7.542887   Topic3  -4.5407  -0.0496
7       fire_brigade   2.000298  10.654765   Topic3  -4.5400  -0.3943
49               day   1.402042   4.524723   Topic3  -4.8954   0.1068
213             need   1.400939   3.209110   Topic3  -4.8962   0.4496

[100 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
177       2  0.526141   argument
18        1  0.996164  attention
99        1  0.996172       blue
111       2  0.790059      child
241       3  0.543389     circle
...     ...       ...        ...
57        3  0.110516    weather
275       2  0.526144       well
68        1  0.996318       work
308       3  0.806794       year
31        1  0.996250      youth

[110 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])